In [ ]:
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import scanorama
import seaborn as sns

In [ ]:
### Scanorama integration

base_path = ### Insert base path
base_path = 'integrated_snn_res.0.4'
ad_ns_int = sc.read_h5ad(f"{base_path}ns_int.h5ad")

ad_ns_int.X = ad_ns_int.raw.X

In [ ]:
sc.pp.normalize_total(ad_ns_int, target_sum = 1e4)
sc.pp.log1p(ad_ns_int)

In [ ]:
sc.pp.highly_variable_genes(ad_ns_int, min_mean = 0.0125, max_mean = 3, min_disp = 0.5, batch_key = "sample")

In [ ]:
var_select = ad_ns_int.var.highly_variable_nbatches >= 22
var_genes = var_select.index[var_select]

In [ ]:
samples = ad_ns_int.obs["sample"].cat.categories.tolist()

In [ ]:
adatas_hvg = [ad_ns_int[ad_ns_int.obs["sample"] == sample][:, var_genes] for sample in samples]

In [ ]:
adatas_scanorama = scanorama.correct_scanpy(adatas_hvg, return_dimred = True) 

In [ ]:
adata_samples = ad.concat(adatas_scanorama)

In [ ]:
sc.pp.neighbors(adata_samples, use_rep = "X_scanorama", key_added = "scanorama")

sc.tl.umap(adata_samples, neighbors_key = "scanorama")

sc.tl.leiden(adata_samples, resolution = 0.6, key_added = "scanorama_clusters", neighbors_key = "scanorama")

In [ ]:
sc.pl.umap(adata_samples, color = "dataset", palette = sns.husl_palette(s = 0.8), legend_loc = "right margin", frameon = False, title = "")

In [ ]:
sc.pl.umap(adata_samples, color = "scanorama_clusters", palette = sns.husl_palette(25, s = 0.8), 
           legend_fontsize = "x-small", legend_fontweight = "medium", legend_loc = "on data", frameon = False, title = "")

In [ ]:
### Plotting

import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 300
%config InlineBackend.figure_formats = ["png"]

In [ ]:
legend_order = list(adata_samples.obs["sample"].unique())
sorted_legend_order = sorted(legend_order, key = str.casefold)
adata_samples.obs["sample"].cat.reorder_categories(sorted_legend_order, inplace = True)
adata_samples.obs["sample"]

In [ ]:
color_palette = sns.husl_palette(38, s = 0.8)
sorted_legend_index = [sorted_legend_order.index(sample) for sample in legend_order]
sorted_color_palette = [color_palette[i] for i in sorted_legend_index]

In [ ]:
sc.pl.umap(adata_samples, color = "sample", palette = sorted_color_palette, 
           legend_fontsize = "x-small", legend_fontweight = "medium", legend_loc = "right margin", frameon = False, title = "")

In [ ]:
save_dir = ### Insert save directory

In [ ]:
adata_samples.write(f"{save_dir}ns_scRNAseq_scanorama_integration.h5ad")

In [ ]:
df_save = pd.DataFrame(adata_samples.obs["scanorama_clusters"])
df_save[["umap_1", "umap_2"]] = adata_samples.obsm["X_umap"]

In [ ]:
df_save.to_csv(f"{save_dir}ns_scRNAseq_scanorama_integration_umap.csv")